# Optimal Solution by Integer Programming

by [Lei You](http://user.it.uu.se/~leiyo378)

In this document, we solve the flexible TTI allocation problem to optimum, by using integer programming solver gurobi.
The formulation of the original problem is as below.

$$
\begin{align}
\max_{\mathbf{x}} \quad & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k} \\
s.t. \quad & \sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k} \quad k\in\mathcal{K}^{(\ell)} \\
           & \sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1 \quad i\in\mathcal{I} \\
           & x_{b,k}\in\{0,1\}\quad b\in\mathcal{B},~k\in\mathcal{K}
\end{align}
$$

We have $x_{b,k}=1$ if and only if physical resource block (PRB) $b$ is allocated to service $k$.

Recall that $\tau_k$ is the maximum tolerant latency of service $k$, and $d_b$ is the end time of the PRB $b$. The constraints for the latency is imposed, by letting $r_{b,k}$ follow the rule:
$$
r_{b,k}=\left\{
\begin{array}{ll}
0 & \text{if } \tau_k-d_b<0 \\
\text{capacity} & \text{otherwise}
\end{array}
\right.
$$

Now let's start :)

In [150]:
import sys
import scipy.io
import numpy
import math
import csv

from gurobipy import *

# set the directory path
import os
folder_name = os.getcwd()

The sets $\mathcal{B}$, $\mathcal{K}^{(\ell)}$, $\mathcal{K}^{(c)}$, $\mathcal{K}$, and $\mathcal{I}$ are read by the following code.

In [151]:
# # set of physical layer blocks (PRBs)
with open('B.csv', 'rb') as f:
    B_csv = csv.reader(f)
    B = list(B_csv)
    B = [item for sublist in B for item in sublist] # flatten list
    B = map(int, map(float, B)) # convert to int
    
# # set of latency services    
with open('Kl.csv', 'rb') as f:
    Kl_csv = csv.reader(f)
    Kl = list(Kl_csv)
    Kl = [item for sublist in Kl for item in sublist] # flatten list
    Kl = map(int, map(float, Kl)) # convert to int
    
# # set of capacity services    
with open('Kc.csv', 'rb') as f:
    Kc_csv = csv.reader(f)
    Kc = list(Kc_csv)
    Kc = [item for sublist in Kc for item in sublist] # flatten list
    Kc = map(int, map(float, Kc)) # convert to int
    
# # set of all services
K = Kl + Kc

# # set of resource units (RUs)
with open('I.csv', 'rb') as f:
    I_csv = csv.reader(f)
    I = list(I_csv)
    I = [item for sublist in I for item in sublist] # flatten list
    I = map(int, map(float, I)) # convert to int

The parameters $\mathbf{r}$, $\mathbf{q}$, and $\mathbf{a}$ are read below.

In [152]:
# # matrix r
with open('r.csv', 'rb') as f:
    r_csv = csv.reader(f)
    r = list(r_csv)
    r = [ map(int,map(float,x)) for x in r] # convert to int

# # vector q, only for Kl
with open('q.csv', 'rb') as f:
    q_csv = csv.reader(f)
    q = list(q_csv)
    q = [item for sublist in q for item in sublist] # flatten list
    q = map(int, map(float, q)) # convert to int
    
# # matrix a
with open('a.csv', 'rb') as f:
    a_csv = csv.reader(f)
    a = list(a_csv)
    a = [ map(int,map(float,x)) for x in a] # convert to int

The model is created below in gurobi.

In [153]:
model = Model('Interger Programming')
model.modelSense = GRB.MAXIMIZE
# model.setParam('OutputFlag', False) # slience output

The variables $x_{b,k}$ ($b\in\mathcal{B}$, $k\in\mathcal{K}$) are added to the model by the following loop. And $\mathbf{x}$ is imposed to be binary.

In [154]:
x = []
for b in B:
    x_b = []
    for k in K:
#         x_b.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=1))
        x_b.append(model.addVar(vtype=GRB.BINARY))
    x.append(x_b)
    
model.update()

The first constraint, $\sum_{b\in\mathcal{B}}r_{b,k}x_{b,k}\geq q_{k}$ for all services $k\in\mathcal{K}^{(\ell)}$, is added below.

In [155]:
for k in Kl:
    model.addConstr( sum(r[b][k]*x[b][k] for b in B) >= q[k] )
model.update()

The second constraint, $\sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}}a_{b,i}x_{b,k}\leq 1$ for all resource units $i\in\mathcal{I}$, is added as follows.

In [156]:
for i in I:
    model.addConstr( sum(a[b][i]*x[b][k] for k in K for b in B ) <= 1 )
model.update()

The objective function $\sum_{b\in\mathcal{B}}\sum_{k\in\mathcal{K}^{(c)}} r_{b,k}x_{b,k}$, is added below.

In [157]:
model.setObjective(
    sum( r[b][k]*x[b][k] for k in Kc for b in B )
)

Now the problem can be solved by the solver:

In [158]:
model.optimize()
print model.Status

Optimize a model with 181 rows, 5490 columns and 23538 nonzeros
Variable types: 0 continuous, 5490 integer (5490 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e+02, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 63 rows and 4392 columns
Presolve time: 0.04s
Presolved: 118 rows, 1098 columns, 5107 nonzeros
Found heuristic solution: objective 1056.0000000
Variable types: 0 continuous, 1098 integer (1098 binary)

Root relaxation: objective 1.986472e+03, 511 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1986.47183    0   13 1056.00000 1986.47183  88.1%     -    0s
H    0     0                    1336.0000000 1986.47183  48.7%     -    0s
H    0     0                    1601.0000000 1986.47183  24.1%     -    0s
     0

In [159]:
def getAssignment(x):
    solK = []
    for k in K:
        sol = [i for i,j in enumerate([int(x[b][k]) for b in B]) if j > 0]
        solK.append(sol)
    return solK

In [160]:
sol_x = [ [ x[b][k].X for k in K] for b in B ]
getAssignment(sol_x)

[[2, 81, 149, 179, 225],
 [92, 105, 131, 143, 218],
 [177, 188, 253, 283, 533, 542, 544],
 [429, 446, 447, 448, 471, 543],
 [26, 55, 68, 529, 530],
 [15, 449, 450, 451, 452],
 [],
 [118, 285],
 [547, 548],
 [255]]

In [161]:
[ sum(int(r[b][k]*sol_x[b][k]) for b in B) for k in K]

[1049, 1049, 1050, 1051, 1050, 945, 0, 440, 350, 197]

In [162]:
q

[1048, 1048, 1048, 1048, 1048]